In [1]:
#import dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import time
import json


In [2]:
#pointing to the directory where chromedriver exists
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
#visiting the page
url = "https://mars.nasa.gov/news/"
browser.visit(url)
#using bs to write it into html
html = browser.html
soup = bs(html,"html.parser")

# NASA Mars News

In [4]:
news_title = soup.find("div",class_="content_title").text
news_p = soup.find("div", class_="article_teaser_body").text
print(f"Title: {news_title}")
print(f"Para: {news_p}")

Title: NASA Announces Landing Site for Mars 2020 Rover
Para: After a five-year search, NASA has chosen Jezero Crater as the landing site for its upcoming Mars 2020 rover mission.


# JPL Mars Space Images - Featured Image

In [5]:
featured_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(featured_image_url)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(2)
browser.click_link_by_partial_text("more info")

In [6]:
# create variable for more info page 
more_info = browser.html
lg_img_Soup = bs(more_info,'html.parser')

# extract img src tag 
lg_img_src = lg_img_Soup.find('figure', class_='lede').find('img')['src']

# assemble URL for large image 
featured_image_url = 'https://www.jpl.nasa.gov' + lg_img_src 

featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA02570_hires.jpg'

# Mars Weather

In [7]:
#mars twitter account
mars_twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_twitter_url)
mars_twitter_html = browser.html
mars_twitter_soup = bs(mars_twitter_html, 'lxml').body

In [8]:
mars_weather = mars_twitter_soup.find('p', class_="tweet-text").text.strip()
mars_weather

'Sol 2233 (2018-11-17), high -4C/24F, low -72C/-97F, pressure at 8.61 hPa, daylight 06:24-18:41'

# Mars Fact

In [9]:
mars_facts_url = 'https://space-facts.com/mars/'
fact_tables = pd.read_html(mars_facts_url)
facts = fact_tables[0]
facts.columns = ['', 'Values']
mars_facts = facts.set_index('', inplace=True)

In [10]:
#Turn dataframe in to html code
mars_facts_html = facts.to_html()
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Values</th>\n    </tr>\n    <tr>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbod

# Mars Hemispheres

In [11]:
url_hem = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_hem)
hemi_list = []

In [12]:
#loop through
for hem in range (4):
    title = browser.find_by_tag('h3')
    title[hem].click()
    
    html = browser.html
    soup = bs(html, 'html.parser')
    
    wide = soup.find("img", class_="wide-image")["src"]
    image_title = soup.find("h2",class_="title").text
    image_url = 'https://astrogeology.usgs.gov'+ wide
    
    dictionary = {"title":image_title,"img_url":image_url}
    hemi_list.append(dictionary)
    
    browser.back()
    time.sleep(5)
hemi_list

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [13]:
browser.quit()